In [1]:
from joblib import load
clf = load('email_predict.joblib')

In [2]:
import jieba
text = '''
亲爱的用户：
为了加强网络安全管理，提高邮件系统的安全性和稳定性，保障收发畅通，为用户提供优质的服务，现即将启用新版系统，有关事项通知如下：
1.用户需登录新邮件系统将原有数据备案进行保存升级。
2.未备案升级数据的用户，系统将其认定为无人使用的账户并停止服务。
3.备案升级后用户名和密码均不变，用户无需修改客户端软件设置。
点此登录完成本次升级
特此通知
2023/8/22
'''
def split_chinese(x):
    return ' '.join(jieba.cut(x))

text = split_chinese(text)
text

'\n 亲爱 的 用户 ： \n 为了 加强 网络安全 管理 ， 提高 邮件系统 的 安全性 和 稳定性 ， 保障 收发 畅通 ， 为 用户 提供 优质 的 服务 ， 现 即将 启用 新版 系统 ， 有关 事项 通知 如下 ： \n 1 . 用户 需 登录 新 邮件系统 将 原有 数据 备案 进行 保存 升级 。 \n 2 . 未备案 升级 数据 的 用户 ， 系统 将 其 认定 为 无人 使用 的 账户 并 停止 服务 。 \n 3 . 备案 升级 后 用户名 和 密码 均 不变 ， 用户 无需 修改 客户端 软件 设置 。 \n 点此 登录 完成 本次 升级 \n 特此通知 \n 2023 / 8 / 22 \n'

In [2]:
import os
from bs4 import BeautifulSoup
from email import policy
from email.parser import BytesParser
import pandas as pd
import jieba

def get_body(message):
    if message.is_multipart():
        return get_body(message.get_payload(0))
    else:
        return message.get_payload(None, True)

def eml_to_dict(eml_path):
    with open(eml_path, 'rb') as f:
        msg = BytesParser(policy=policy.default).parse(f)

    # get headers
    data = dict(msg.items())

    # get body
    body = get_body(msg)
    if body is not None:
        try:
            data['body'] = body.get_content()
        except AttributeError:
            data['body'] = body
        # get text from html
        data['body'] = BeautifulSoup(data['body'], "html.parser").text.strip()
    data['email_id'] = os.path.basename(eml_path).split('.')[0]
    return data

# dataset path
eml_dir = os.path.join('mail')

eml_files = [os.path.join(eml_dir, filename) for filename in os.listdir(eml_dir) if filename.endswith(".eml")]

data = [eml_to_dict(eml_file) for eml_file in eml_files]
df = pd.DataFrame(data)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'mail'

In [ ]:
data = df[['email_id','Subject', 'body']]
data['Subject'] = data['Subject'].str.replace(' ', '').str.replace('〉', '').str.replace('\n', '').str.replace('\r', '').str.replace('\t', '').str.replace('\xa0', ' ').str.replace('\ufeff', ' ')
data['body'] = data['body'].str.replace('\n', '').str.replace('\r', '').str.replace('\t', '').str.replace('\xa0', ' ').str.replace('\ufeff', ' ')

In [ ]:
data['Message'] = data['Subject'] + ' ' + data['body']
data['Message'] = data['Message'].apply(lambda x: ' '.join(jieba.cut(x)))

In [ ]:
data['Predict'] = clf.predict(data['Message'])

In [ ]:
data[['email_id','Subject','body', 'Predict']]

In [ ]:
with open('answer.txt', 'w') as f:
    for index, row in data.iterrows():
        email_id = str(row['email_id'])
        predict = str(row['Predict'])
        f.write(email_id + ',' + predict + '\n')